In [148]:
import numpy as np
from scipy.stats import betaprime
from scipy.special import beta
%matplotlib tk
import matplotlib.pyplot as plt
import random
import matplotlib.animation as animation

In [149]:
def simulador(t, t0, sigma, omega):
    """
    Funcion que realiza la gráfica de un pulso base
    t       -> Array NP de los tiempos a sondear
    t0      -> Valor central del pico
    sigma   -> Ancho del pico
    omega   -> Frecuencia de oscilacion
    """
    #Determino el valor de A
    A = 1 / (sigma * np.sqrt(2 * np.pi))
    pulso = A * np.exp(-(t - t0) ** 2 / (2 * sigma ** 2)) * np.cos(omega * (t - t0 + sigma))
    return pulso

def gaussian(mean, sigma):
    return np.random.normal(mean, sigma)

In [150]:
#Obtengo los puntos de tiempo para el muestreo (1000 puntos)
t = np.linspace(0, 10, 1000)

#Creo una funcion para generar los números aleatorios
def aleatorios():
    #Calculo los tiempos donde se van a generar los picos con distribución beta prime
    # Definir los parámetros de la distribución
    t0s=[1.8, 2.8, 5]
    a = 4
    b = 2
    sigma_t = 0.15
    taux=[]
    for t0x in t0s:
        t = betaprime.rvs(a, b, loc=t0x, scale=sigma_t, size=1)
        taux.append(t[0])
    tiempos=np.array(taux)

    #Calculo los anchos de los pulsos(sigmas)

    sigmas=np.array([np.random.normal(0.05, 0.005) for _ in range(3)]) 

    #Calculo las frecuencias de oscilacion
    omegas=np.array([np.random.normal(5*np.pi, 0.05) for _ in range(3)])
    
    #Probabilidades de los picos
    px=np.random.random(size=3)
    prob=np.array([0.3, 0.1, 0.6])
    aciertos=np.zeros(3)
    for x in range(3):
        if px[x]<=prob[x]:aciertos[x]=1
    return tiempos, sigmas, omegas, aciertos

In [151]:
# Señal completa
def senialcomp(muestras):
    tiem, sig, ome, match = aleatorios()
    #obtengo el ruido absoluto
    ruido_abs = np.random.normal(0, 0.2, size=1000)
    #Obtengo el ruido relativo
    ruido_rel = np.random.normal(1, 0.1, size=1000)
    
    
    #fondo
    fondo=(2-muestras) / 100
    senial1=match[0]*simulador(t, tiem[0],sig[0],ome[0])
    senial2=match[1]*simulador(t, tiem[1],sig[1],ome[1])
    senial3=match[2]*simulador(t, tiem[2],sig[2],ome[2])
    signal=fondo+(senial1+senial2+senial3)*ruido_rel+ruido_abs
    return signal

# Animación
fig, ax = plt.subplots()
ax.set_xlim(0, 10)
ax.set_ylim(-5, 10)
#line, = ax.plot(t[0], signal[0], lw=2)
line, = plt.plot([], [])
#def init():
#    line.set_data([], [])
 #   return line,

def animate(i,L):
    #gobal t
    sig=senialcomp(t)
    #line.set_data(t[:i], signal[:i])
    L.set_data(t, sig)
    return L,

repeticiones=100
animacion=animation.FuncAnimation(fig, animate,repeticiones,fargs=(line,), interval=300, blit=True)
plt.show()